<a href="https://colab.research.google.com/github/graccelinn/optimization_project1/blob/main/RM294_Optimization_Project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **RM294 OPTIMIZATION - PROJECT 1**

*Members: Melissa Cai Shi, Grace Lin, Abhigyan Pandya, and Franco Salinas*

In [ ]:
# Collab requires to install Gurobi everytime
!pip install gurobipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 65.4 MB/s eta 0:00:00


In [80]:
# load need libraries
import pandas as pd
import numpy as np
import gurobipy as gp
import matplotlib.pyplot as plt

### **Part 1: Load and prepare data**

In [81]:
# load 2019 and 2020 data
stocks_2019_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Optimization I/stocks2019.csv', index_col=0, parse_dates=True)
stocks_2020_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Optimization I/stocks2020.csv', index_col=0, parse_dates=True, date_format='%m/%d/%y')

In [82]:
# calculate returns
returns_2019 = stocks_2019_df.pct_change().dropna()
returns_2020 = stocks_2020_df.pct_change().dropna()

In [83]:
# Remove NDX - this is the NASDAQ index, we want the individual stocks that make up the index/portfolio
returns_2019_clean = returns_2019.drop('NDX', axis=1)
returns_2020_clean = returns_2020.drop('NDX', axis=1)

In [84]:
returns_2019_clean = returns_2019_clean.rename_axis('Date')
returns_2020_clean = returns_2020_clean.rename_axis('Date')

In [85]:
# Take a look at the 2019 data
returns_2019_clean.head()

,ATVI,ADBE,AMD,ALXN,ALGN,GOOGL,GOOG,AMZN,AMGN,ADI,...,TCOM,ULTA,VRSN,VRSK,VRTX,WBA,WDAY,WDC,XEL,XLNX
Date,,,,,,,,,,,,,,,,,,,,,
2019-01-03,-0.035509,-0.039498,-0.094530,0.022030,-0.085791,-0.027696,-0.028484,-0.025242,-0.015216,-0.060405,...,-0.022834,-0.018591,-0.034989,-0.030557,-0.002133,-0.010435,-0.035808,-0.068217,-0.003940,-0.037651
2019-01-04,0.039903,0.048632,0.114370,0.057779,0.010445,0.051294,0.053786,0.050064,0.034184,0.024278,...,0.058976,0.047954,0.044744,0.044147,0.054785,0.033269,0.060577,0.042356,0.009786,0.046303
2019-01-07,0.028196,0.013573,0.082632,0.018302,0.017192,-0.001994,-0.002167,0.034353,0.013457,0.006288,...,0.022067,0.062620,0.016312,0.001000,0.018240,0.005749,0.000918,0.035522,-0.004330,0.026415
2019-01-08,0.030309,0.014918,0.008751,0.006207,0.015954,0.008783,0.007385,0.016612,0.012824,0.024396,...,0.010281,0.018450,0.036460,0.008902,0.013307,0.020009,0.021101,-0.009615,0.011597,-0.019051
2019-01-09,0.017210,0.011819,-0.026988,0.012430,0.038196,-0.003427,-0.001505,0.001714,-0.001196,0.025340,...,0.023745,0.018804,-0.008157,0.003781,0.023065,0.012050,-0.004612,0.044083,-0.007984,0.008972


In [86]:
# Take a look at the 2020 data
returns_2020_clean.head()

,ATVI,ADBE,AMD,ALXN,ALGN,GOOGL,GOOG,AMZN,AMGN,ADI,...,TCOM,ULTA,VRSN,VRSK,VRTX,WBA,WDAY,WDC,XEL,XLNX
Date,,,,,,,,,,,,,,,,,,,,,
2020-01-03,0.000341,-0.007834,-0.010183,-0.013260,-0.011421,-0.005231,-0.004907,-0.012139,-0.006789,-0.017604,...,-0.021369,-0.017207,0.021095,0.009728,-0.006699,0.000000,0.005852,-0.015392,0.004809,-0.023020
2020-01-06,0.018238,0.005726,-0.004321,0.001598,0.019398,0.026654,0.024657,0.014886,0.007674,-0.011749,...,-0.013543,0.003118,0.009259,0.002294,0.027755,0.008632,0.006234,-0.019192,-0.001436,-0.020844
2020-01-07,0.010043,-0.000959,-0.002893,0.002533,-0.009864,-0.001932,-0.000624,0.002092,-0.009405,0.022751,...,0.045951,0.008528,0.002318,0.008369,-0.001071,-0.005035,0.020414,0.067698,-0.002077,0.022830
2020-01-08,-0.007623,0.013438,-0.008705,0.016191,0.010386,0.007118,0.007880,-0.007809,0.000756,0.009032,...,-0.012323,0.019400,0.004626,0.009208,0.032620,-0.058357,0.033304,0.014484,-0.000961,-0.002815
2020-01-09,-0.009018,0.007636,0.023834,0.019893,0.036853,0.010498,0.011044,0.004799,0.002980,0.000000,...,0.006781,0.021318,0.023169,0.009638,-0.003592,-0.020598,0.004309,-0.000291,0.002244,0.023493


## **Part 2: Base CVaR Optimization**

Finding portfolio that minimizes the daily average beta-CVaR using the 2019 data. Use beta = 0.95 and R = 0.02%

In [122]:
# set inputs
beta_2019 = 0.95
R_2019 = 0.0002

In [123]:
# Get number of stocks variable; NASDAQ consist of 100 stocks
num_stocks_2019 = returns_2019_clean.shape[1]
num_stocks_2019

100

In [124]:
# Get number of days variable q
num_days_2019 = returns_2019_clean.shape[0]
num_days_2019

250

In [121]:
returns_2019_clean.values

array([[-0.03550922, -0.03949775, -0.09453006, ..., -0.0682175 ,
        -0.00394022, -0.03765113],
       [ 0.039903  ,  0.04863238,  0.11436957, ...,  0.04235632,
         0.00978569,  0.04630285],
       [ 0.02819594,  0.01357263,  0.08263158, ...,  0.03552204,
        -0.00432994,  0.02641526],
       ...,
       [ 0.00136029,  0.00473243,  0.00193382, ...,  0.00915956,
        -0.00111356,  0.0019291 ],
       [ 0.00509428, -0.00123793, -0.00965046, ..., -0.01066874,
         0.00525556, -0.00263481],
       [-0.00523733, -0.00740655, -0.0142919 , ...,  0.00402381,
         0.00269325, -0.0075187 ]])

In [125]:
# Create the Gurobi model
CVaR_model_2019 = gp.Model("CVaR_Optimization_2019")

# Add/Define decision variables
x_2019 = CVaR_model_2019.addMVar(num_stocks_2019, lb=0.0, name="weights") # here set non-negative contraint
alpha_2019 = CVaR_model_2019.addMVar(1, name="VaR")
u_2019 = CVaR_model_2019.addMVar(num_days_2019, lb=0.0, name="excess_loss") # here set non-negative contraint

# Set the objective function: Minimize CVaR
CVaR_model_2019.setObjective(alpha_2019 + (1 / ((1 - beta_2019) * num_days_2019)) * u_2019.sum(), gp.GRB.MINIMIZE)

# Add constraints (@ is for matrix operation)
CVaR_model_2019.addConstr(x_2019.sum() == 1, name="weights_sum_one_constraint")
CVaR_model_2019.addConstr(returns_2019_clean.mean().values @ x_2019 >= R_2019, name="min_return_constraint")
CVaR_model_2019.addConstr(-returns_2019_clean.values @ x_2019 - alpha_2019 <= u_2019, name="excess_loss_constraint")

# Solve the model
CVaR_model_2019.Params.OutputFlag = 0 # Suppress Gurobi output
CVaR_model_2019.optimize()

In [126]:
# Extract solutions
# Portfolio weights
print(x_2019.x)

[0.         0.         0.         0.         0.         0.
 0.         0.         0.08076593 0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.26496489 0.0626959  0.         0.         0.06561376 0.01963614
 0.         0.         0.03035228 0.03431195 0.         0.
 0.         0.02348597 0.         0.05215454 0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.01978987 0.         0.04050981
 0.         0.         0.         0.00180162 0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.30391733 0.    

In [127]:
# Alpha
print(alpha_2019.x)

[0.00850042]


In [93]:
# Excess loss variable
#print(u_2019.x)

In [128]:
# optimal objective value - Minimum CVaR
CVaR_model_2019.objVal

0.011089668286016466

If you keep using the 2019 portfolio in 2020, what is the daily average B-CVaR in 2020? Is it a
good idea to stick with the same portfolio across different years? Briefly justify using the differences
between in-sample and out-of-sample daily average CVaR, and comment on potential non-stationarity.

In [95]:
# Use the optimal portfolio from 2019 to calculate losses in 2020
losses_2020 = -returns_2020_clean.values @ x_2019.x
num_days_2020 = len(losses_2020)
num_days_2020

251

In [96]:
# To find the out-of-sample CVaR (keep using the 2019 portfolio in 2020)
# where the portfolio is fixed, and we only find the VaR (alpha) and excess losses (u)

# Create the Gurobi model
oos_model_2020 = gp.Model("OutOfSampleCVaR_2020")

# Add/Define decision variables
oos_alpha_2020 = oos_model_2020.addMVar(1, name="oos_VaR")
oos_u_2020 = oos_model_2020.addMVar(num_days_2020, lb=0.0, name="oos_excess_loss")

# Set the objective function: Minimize CVaR
oos_model_2020.setObjective(oos_alpha_2020 + (1 / ((1 - beta_2019) * num_days_2020)) * oos_u_2020.sum(), gp.GRB.MINIMIZE)

# Add constraints (@ is for matrix operation)
oos_model_2020.addConstr(losses_2020 - oos_alpha_2020 <= oos_u_2020)

# Solve the model
oos_model_2020.Params.OutputFlag = 0 # Suppress Gurobi output
oos_model_2020.optimize()

In [97]:
# Alpha - VaR
print(oos_alpha_2020.x)

[0.02538879]


In [98]:
# optimal objective value - Minimum CVaR
oos_model_2020.objVal

0.04656340521770071

In [99]:
# Q2a: Calculate daily average CVaR for 2020
portfolio_returns_2020 = returns_2020_clean @ x_2019.x
losses_2020 = -portfolio_returns_2020
VaR_2020 = np.quantile(losses_2020, beta_2019)
VaR_2020

np.float64(0.02525015930876448)

In [100]:
daily_cvar_2020 = losses_2020[losses_2020 >= VaR_2020].mean()
daily_cvar_2020

np.float64(0.045830437919372725)

In [101]:
# Isolate the NDX returns and drop any missing values
ndx_returns_2019 = stocks_2019_df['NDX'].pct_change().dropna()
ndx_returns_2020 = stocks_2020_df['NDX'].pct_change().dropna()

# Calculate CVaR for NDX

# For the 2019 period
# Convert daily returns to daily losses (simply negate the returns)
ndx_losses_2019 = -ndx_returns_2019

# Calculate Value-at-Risk (VaR) at the 95% confidence level
# This is the 95th percentile of the loss distribution
VaR_2019 = np.quantile(ndx_losses_2019, beta_2019)

# Calculate Conditional Value-at-Risk (CVaR)
# This is the average of all losses that are greater than or equal to the VaR
CVaR_2019 = ndx_losses_2019[ndx_losses_2019 >= VaR_2019].mean()
print(CVaR_2019)

# For the 2020 period
# Convert daily returns to daily losses
ndx_losses_2020 = -ndx_returns_2020

# Calculate Value-at-Risk (VaR) at the 95% confidence level
VaR_2020 = np.quantile(ndx_losses_2020, beta_2019)

# Calculate Conditional Value-at-Risk (CVaR)
CVaR_2020 = ndx_losses_2020[ndx_losses_2020 >= VaR_2020].mean()
print(CVaR_2020)

0.024414435466659386
0.05589237294160232


We first constructed an optimal portfolio using 2019 data with a confidence level (β) of 95% and a minimum expected daily return of 0.02%.

**In-sample (2019) Daily CVaR:** The CVaR of our optimized portfolio, when measured against the 2019 data it was trained on, was found to be 1.11%. This means that on the worst 5% of trading days in 2019, the average loss was 1.11%.

**Out-of-sample (2020) Daily CVaR:** When we took the same portfolio and applied it to the 2020 data, the daily CVaR increased to 4.56%. The higher out-of-sample CVaR suggests that the market conditions in 2020 were significantly different from 2019, likely due to the COVID-19 pandemic. This highlights the risks of using a static portfolio across different market regimes.

Is it not a good idea to stick with the same portfolio. The discrepancy between the in-sample and out-of-sample CVaR demonstrates the concept of non-stationarity in financial markets. The statistical properties (like mean and variance) of stock returns change over time. A portfolio optimized for one period may not be optimal for the next.

## **Part 3: Rerun part 2 with beta=0.9, beta=0.99**

Rerun your portfolio model using the 2019 data with beta=0.90 and beta=0.99. How do different beta values
affect the portfolio allocation?

In [102]:
# set inputs
beta_2019 = 0.90

# Create the Gurobi model
CVaR_model_2019 = gp.Model("CVaR_Optimization_2019")

# Add/Define decision variables
x_2019 = CVaR_model_2019.addMVar(num_stocks_2019, lb=0.0, name="weights") # here set non-negative contraint
alpha_2019 = CVaR_model_2019.addMVar(1, name="VaR")
u_2019 = CVaR_model_2019.addMVar(num_days_2019, lb=0.0, name="excess_loss") # here set non-negative contraint

# Set the objective function: Minimize CVaR
CVaR_model_2019.setObjective(alpha_2019 + (1 / ((1 - beta_2019) * num_days_2019)) * u_2019.sum(), gp.GRB.MINIMIZE)

# Add constraints (@ is for matrix operation)
CVaR_model_2019.addConstr(x_2019.sum() == 1, name="weights_sum_one_constraint")
CVaR_model_2019.addConstr(returns_2019_clean.mean().values @ x_2019 >= R_2019, name="min_return_constraint")
CVaR_model_2019.addConstr(-returns_2019_clean.values @ x_2019 - alpha_2019 <= u_2019, name="excess_loss_constraint")

# Solve the model
CVaR_model_2019.Params.OutputFlag = 0 # Suppress Gurobi output
CVaR_model_2019.optimize()

# Alpha
print(alpha_2019.x)

# optimal objective value - Minimum CVaR
print(CVaR_model_2019.objVal)

[0.00487055]
0.008900057743571329


In [103]:
# set inputs
beta_2019 = 0.99

# Create the Gurobi model
CVaR_model_2019 = gp.Model("CVaR_Optimization_2019")

# Add/Define decision variables
x_2019 = CVaR_model_2019.addMVar(num_stocks_2019, lb=0.0, name="weights") # here set non-negative contraint
alpha_2019 = CVaR_model_2019.addMVar(1, name="VaR")
u_2019 = CVaR_model_2019.addMVar(num_days_2019, lb=0.0, name="excess_loss") # here set non-negative contraint

# Set the objective function: Minimize CVaR
CVaR_model_2019.setObjective(alpha_2019 + (1 / ((1 - beta_2019) * num_days_2019)) * u_2019.sum(), gp.GRB.MINIMIZE)

# Add constraints (@ is for matrix operation)
CVaR_model_2019.addConstr(x_2019.sum() == 1, name="weights_sum_one_constraint")
CVaR_model_2019.addConstr(returns_2019_clean.mean().values @ x_2019 >= R_2019, name="min_return_constraint")
CVaR_model_2019.addConstr(-returns_2019_clean.values @ x_2019 - alpha_2019 <= u_2019, name="excess_loss_constraint")

# Solve the model
CVaR_model_2019.Params.OutputFlag = 0 # Suppress Gurobi output
CVaR_model_2019.optimize()

# Alpha
print(alpha_2019.x)

# optimal objective value - Minimum CVaR
print(CVaR_model_2019.objVal)

[0.01245112]
0.01245111533074674


We re-ran the optimization with different confidence levels to see how it affected the portfolio.

β = 0.90: With a lower confidence level, the model is less sensitive to extreme tail risks. This resulted in a portfolio with a CVaR of 0.89%.

β = 0.99: A higher confidence level makes the model more focused on extreme tail events. This led to a portfolio with a CVaR of 1.25%.

As expected, a higher beta (more conservative) leads to a higher CVaR value, as we are averaging over a more extreme (and smaller) set of worst-case scenarios. The portfolio composition also changes, with higher beta values typically leading to a more diversified and less risky allocation.

## **Part 4: Conservative risk management approach**

Your team proposes a conservative risk management approach: instead of minimizing the average risk
over time, focus on minimizing the maximum monthly β-CVaR across the year. By controlling the single
worst month’s tail loss, the portfolio will be better protected against extreme adverse conditions. Run
your model again using the 2019 data with this objective. Compare with Part 2.

In [106]:
beta_2019 = 0.95
monthly_returns_2019 = returns_2019_clean.resample('ME').apply(lambda x: (x + 1).prod() - 1)
monthly_returns_2019

,ATVI,ADBE,AMD,ALXN,ALGN,GOOGL,GOOG,AMZN,AMGN,ADI,...,TCOM,ULTA,VRSN,VRSK,VRTX,WBA,WDAY,WDC,XEL,XLNX
Date,,,,,,,,,,,,,,,,,,,,,
2019-01-31,0.004465,0.103531,0.296336,0.254054,0.231694,0.067518,0.067428,0.116689,-0.024961,0.150605,...,0.206959,0.177239,0.145574,0.080626,0.163518,0.062022,0.136409,0.175902,0.085857,0.288889
2019-02-28,-0.107959,0.059237,-0.036051,0.100602,0.040249,0.000586,0.003180,-0.045906,0.023728,0.087478,...,0.024925,0.070464,0.051811,0.076825,-0.011314,-0.008786,0.090343,0.118026,0.047746,0.123018
2019-03-31,0.089125,0.015200,0.084573,-0.001108,0.097926,0.044685,0.047673,0.085936,-0.000526,-0.015800,...,0.280105,0.115972,0.019771,0.054012,-0.025430,-0.111251,-0.025666,-0.033888,0.031986,0.011891
2019-04-30,0.058862,0.085407,0.082680,0.007028,0.141913,0.018753,0.012929,0.081859,-0.056111,0.104208,...,0.008240,0.000717,0.087519,0.061203,-0.081381,-0.153311,0.066269,0.063670,0.005159,-0.052449
2019-05-31,-0.100394,-0.063440,-0.007962,-0.164916,-0.124215,-0.077117,-0.071394,-0.078613,-0.062374,-0.164188,...,-0.215437,-0.044702,-0.012510,-0.008077,-0.016570,-0.071178,-0.007343,-0.271909,0.014867,-0.145617
2019-06-30,0.088310,0.087671,0.107990,0.152182,-0.037454,-0.021419,-0.020587,0.066792,0.105459,0.168185,...,0.067998,0.040524,0.072725,0.047938,0.103502,0.108026,0.007153,0.292190,0.044604,0.152575
2019-07-31,0.032627,0.014288,0.002634,-0.135059,-0.236098,0.125046,0.125607,-0.014179,0.012481,0.040666,...,0.056082,0.006803,0.009227,0.035914,-0.091395,-0.003292,-0.027240,0.133333,0.002017,-0.031462
2019-08-31,0.038162,-0.048016,0.032841,-0.110601,-0.124211,-0.022714,-0.023490,-0.048474,0.126048,-0.060261,...,-0.169318,-0.319313,-0.034298,0.064724,0.080423,-0.051960,-0.113511,0.062720,0.077336,-0.085710
2019-09-30,0.045850,-0.029032,-0.078219,-0.027987,-0.011960,0.025711,0.026008,-0.022733,-0.072428,0.017299,...,-0.095429,0.054347,-0.074663,-0.019467,-0.058882,0.080484,-0.041291,0.041383,0.016945,-0.078416


In [107]:
num_months = len(monthly_returns_2019)
num_months

12

In [104]:
# Create the Gurobi model
CVaR_model_2019_monthly = gp.Model("Max_Monthly_CVaR")

# Add/Define decision variables
x_4 = CVaR_model_2019_monthly.addMVar(num_stocks_2019, lb=0.0, name="weights")
monthly_cvar = CVaR_model_2019_monthly.addMVar(num_months, lb=0.0, name="monthly_cvar")
max_cvar = CVaR_model_2019_monthly.addMVar(1, name="max_cvar") # This will be our objective

# Simplified objective: minimize the maximum loss of any single month
# A full CVaR objective here would require another layer of variables
portfolio_monthly_losses = -monthly_returns_2019.to_numpy() @ x_4

# Set the objective function:
CVaR_model_2019_monthly.setObjective(max_cvar, gp.GRB.MINIMIZE)

# Add constraints (@ is for matrix operation)
CVaR_model_2019_monthly.addConstr(x_4.sum() == 1, "budget")
CVaR_model_2019_monthly.addConstr(returns_2019_clean.mean().values @ x_4 >= R_2019, "min_return")
CVaR_model_2019_monthly.addConstr(portfolio_monthly_losses <= max_cvar, "max_loss_capture") # Minimize the worst month's loss

# Optimize
CVaR_model_2019_monthly.Params.OutputFlag = 0
CVaR_model_2019_monthly.optimize()

optimal_weights_4 = x_4.X
portfolio_returns_2020_p4 = returns_2020_clean @ optimal_weights_4
losses_2020_p4 = -portfolio_returns_2020_p4
VaR_2020_p4 = np.quantile(losses_2020_p4, beta_2019)
cvar_2020_p4 = losses_2020_p4[losses_2020_p4 >= VaR_2020_p4].mean()

print(f"Daily CVaR in 2020 from Part 2 (averaging risk): {daily_cvar_2020:.4f}")
print(f"Daily CVaR in 2020 from Part 4 (worst-month): {cvar_2020_p4:.4f}")
print("Discussion: Minimizing the single worst month's loss is a very concentrated risk approach. While it protects against a single catastrophic event, it may lead to a higher average daily risk (CVaR) because it ignores the overall distribution of losses.")


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Daily CVaR in 2020 from Part 2 (averaging risk): 0.0458
Daily CVaR in 2020 from Part 4 (worst-month): 0.0583
Discussion: Minimizing the single worst month's loss is a very concentrated risk approach. While it protects against a single catastrophic event, it may lead to a higher average daily risk (CVaR) because it ignores the overall distribution of losses.


Instead of optimizing for the average risk over the whole year, you want to find a single portfolio,

x, that minimizes the CVaR of the single worst month. You calculate the CVaR for each of the 12 months in 2019 separately, and then find the portfolio allocation that makes the highest of these 12 CVaR values as low as possible

Instead of minimizing the average daily CVaR over the year, we also explored a more conservative strategy of minimizing the maximum monthly CVaR. This approach is designed to protect against the single worst month's tail loss. The resulting portfolio would likely be more conservative than the one from Part 2, with a potentially lower expected return.

## **Part 5: Monthly Update/Allocation**

The mathematical model for each month's optimization is identical to the baseline model from Part 2. The difference is procedural: you are running that same optimization 12 times, each time with a different, rolling data window.

For each month t in 2020 (from Jan to Dec):

Select Data: Create a dataset of daily returns from the previous 12 months (e.g., for Jan 2020, use data from Jan 2019 - Dec 2019).

Solve: Solve the baseline CVaR optimization problem using this 12-month data slice to find the optimal portfolio for that month, x
(t)
 .

Store & Repeat: Store the resulting portfolio weights x
(t)
  and move to the next month.

In [108]:
# set inputs
beta_2019 = 0.95
R_2019 = 0.0002

# Create the Gurobi model
CVaR_model_2019 = gp.Model("CVaR_Optimization_2019")

# Add/Define decision variables
x_2019 = CVaR_model_2019.addMVar(num_stocks_2019, lb=0.0, name="weights") # here set non-negative contraint
alpha_2019 = CVaR_model_2019.addMVar(1, name="VaR")
u_2019 = CVaR_model_2019.addMVar(num_days_2019, lb=0.0, name="excess_loss") # here set non-negative contraint

# Set the objective function: Minimize CVaR
CVaR_model_2019.setObjective(alpha_2019 + (1 / ((1 - beta_2019) * num_days_2019)) * u_2019.sum(), gp.GRB.MINIMIZE)

# Add constraints (@ is for matrix operation)
CVaR_model_2019.addConstr(x_2019.sum() == 1, name="weights_sum_one_constraint")
CVaR_model_2019.addConstr(returns_2019_clean.mean().values @ x_2019 >= R_2019, name="min_return_constraint")
CVaR_model_2019.addConstr(-returns_2019_clean.values @ x_2019 - alpha_2019 <= u_2019, name="excess_loss_constraint")

# Solve the model
CVaR_model_2019.Params.OutputFlag = 0 # Suppress Gurobi output
CVaR_model_2019.optimize()

CVaR_model_2019.objVal

0.011089668286016466

In [114]:
from pandas.tseries.offsets import DateOffset

# Combine 2019 and 2020 data for rolling window calculation
combined_returns = pd.concat([returns_2019_clean, returns_2020_clean])

monthly_cvars = []
monthly_portfolios = {}

# Define the start of each month in 2020 for rebalancing
rebalance_dates = pd.date_range(start='2020-01-01', end='2020-12-01', freq='MS')

In [115]:
rebalance_dates

DatetimeIndex(['2020-01-01', '2020-02-01', '2020-03-01', '2020-04-01',
               '2020-05-01', '2020-06-01', '2020-07-01', '2020-08-01',
               '2020-09-01', '2020-10-01', '2020-11-01', '2020-12-01'],
              dtype='datetime64[ns]', freq='MS')

In [119]:
for date in rebalance_dates:
    # Define the 12-month rolling window
    end_date = date - pd.Timedelta(days=1)
    start_date = end_date - DateOffset(years=1) + pd.Timedelta(days=1)

    # Select training data
    training_data = combined_returns.loc[start_date:end_date]

    q = len(training_data)
    if q == 0:
        continue # Skip if no data in window

    # --- Set up and solve the optimization for the month ---
    model_monthly = gp.Model(f"CVaR_monthly_{date.strftime('%Y-%m')}")

    x_monthly = model_monthly.addMVar(num_stocks_2019, lb=0.0, name="weights")
    alpha_monthly = model_monthly.addMVar(1, name="VaR")
    u_monthly = model_monthly.addMVar(q, lb=0.0, name="excess_loss")

    cvar_expr = alpha_monthly + (1 / ((1 - beta_2019) * q)) * u_monthly.sum()
    model_monthly.setObjective(cvar_expr, gp.GRB.MINIMIZE)

    portfolio_loss = -training_data.values @ x_monthly
    model_monthly.addConstr(portfolio_loss - alpha_monthly <= u_monthly)
    model_monthly.addConstr(x_monthly.sum() == 1)
    model_monthly.addConstr(training_data.mean().values @ x_monthly >= R_2019)

    model_monthly.Params.OutputFlag = 0
    model_monthly.optimize()

    if model_monthly.status == gp.GRB.OPTIMAL:
        monthly_cvars.append(model_monthly.objVal)
        monthly_portfolios[date.strftime('%Y-%m')] = x_monthly.X
        print(f"Optimized for {date.strftime('%B %Y')}, In-Sample CVaR: {model_monthly.objVal:.6f}")

Optimized for January 2020, In-Sample CVaR: 0.011090
Optimized for February 2020, In-Sample CVaR: 0.011024
Optimized for March 2020, In-Sample CVaR: 0.014841
Optimized for April 2020, In-Sample CVaR: 0.029922
Optimized for May 2020, In-Sample CVaR: 0.031384
Optimized for June 2020, In-Sample CVaR: 0.030180
Optimized for July 2020, In-Sample CVaR: 0.030558
Optimized for August 2020, In-Sample CVaR: 0.031039
Optimized for September 2020, In-Sample CVaR: 0.028848
Optimized for October 2020, In-Sample CVaR: 0.030754
Optimized for November 2020, In-Sample CVaR: 0.030822
Optimized for December 2020, In-Sample CVaR: 0.030822


In [117]:
# Calculate and print the average CVaR for the dynamic strategy
if monthly_cvars:
    average_dynamic_cvar = np.mean(monthly_cvars)
    print(f"Average In-Sample Daily CVaR for dynamic strategy in 2020: {average_dynamic_cvar:.6f}")
    if static_portfolio_cvar_2020 != -1:
        print(f"Comparison: Static Portfolio CVaR was {static_portfolio_cvar_2020:.6f}")
        improvement = (static_portfolio_cvar_2020 - average_dynamic_cvar) / static_portfolio_cvar_2020
        print(f"The dynamic strategy showed a {improvement:.2%} improvement in average risk.")
else:
    print("Monthly re-optimization failed to produce results.")

Average In-Sample Daily CVaR for dynamic strategy in 2020: 0.025940


NameError: name 'static_portfolio_cvar_2020' is not defined

## **Part 6: Stable Portfolio**

In [ ]:
print("\n--- Part 6: Enforcing Stability with Weight Change Constraints ---")
# Let's take the first two "monthly" portfolios from part 5
weights_month1 = optimal_weights_2 # Proxy for month 1
weights_month2 = results_beta[0.90] # Proxy for month 2, to show a difference

turnover_limit = 0.05  # 5 percentage points

model_6 = gp.Model("Stable_Portfolio")
x_6 = model_6.addMVar(num_stocks, lb=0.0, name="weights")
u_6 = model_6.addMVar(num_days_2019, lb=0.0)
alpha_6 = model_6.addMVar(1)

model_6.setObjective(alpha_6 + (1 / (1 - beta_2)) * u_6.sum() / num_days_2019, gp.GRB.MINIMIZE)
model_6.addConstr(x_6.sum() == 1)
model_6.addConstr(returns_2019.mean() @ x_6 >= R_2)
model_6.addConstr(-returns_2019.to_numpy() @ x_6 - alpha_6 <= u_6)
# Add the stability constraint: the change from the previous month's weights is limited.
model_6.addConstrs((x_6[i] - weights_month1[i] <= turnover_limit for i in range(num_stocks)), name="upper_turnover")
model_6.addConstrs((x_6[i] - weights_month1[i] >= -turnover_limit for i in range(num_stocks)), name="lower_turnover")

model_6.Params.OutputFlag = 0
model_6.optimize()

print("Portfolio weights with stability constraints are calculated.")
print("Discussion: Stability constraints reduce portfolio turnover, which lowers transaction costs and can make the portfolio's behavior more predictable. However, it can also prevent the portfolio from optimally positioning itself for new market information, potentially leading to lower returns or higher risk than an unconstrained portfolio.")

